<a href="https://colab.research.google.com/github/APYazdi/Crypto-Sentiment-Analysis/blob/main/FinBERT_Crypto_Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Getting Dependecies**


In [ ]:
!pip install transformers

     |████████████████████████████████| 4.0 MB 12.2 MB/s 
     |████████████████████████████████| 6.6 MB 44.4 MB/s 
     |████████████████████████████████| 77 kB 3.7 MB/s 
     |████████████████████████████████| 596 kB 26.8 MB/s 
     |████████████████████████████████| 880 kB 47.0 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=ce72e3bd06fb20b6f8c1780b1ce06165ab3297656c0cd27830f191ae237843a4
  Stored in directory: /root/.cache/pip/wheels/87/39/dd/a83eeef36d0bf98e7a4d1933a4ad2d660295a40613079bafc9
Successfully built sacremoses
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
!pip install pygooglenews --upgrade

     |████████████████████████████████| 63 kB 1.9 MB/s 
     |████████████████████████████████| 1.2 MB 48.0 MB/s 
     |████████████████████████████████| 362 kB 39.5 MB/s 
     |████████████████████████████████| 128 kB 56.9 MB/s 
  Created wheel for feedparser: filename=feedparser-5.2.1-py3-none-any.whl size=44952 sha256=bd5feee73edcba4a79e773038a5c67718be434c1625e2f8565b1f70a653c35ff
  Stored in directory: /root/.cache/pip/wheels/29/bf/46/b4a597d435d3aee6c2fa583824897336d65abf13ebe3405b70
Successfully built feedparser
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
  Attempting uninstall: beautifulsoup4
    Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the

In [ ]:
!pip install wandb

     |████████████████████████████████| 1.8 MB 16.1 MB/s 
     |████████████████████████████████| 181 kB 56.9 MB/s 
     |████████████████████████████████| 144 kB 11.1 MB/s 
     |████████████████████████████████| 63 kB 1.8 MB/s 
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8806 sha256=b86668dae8b994ac1cb22d7baf672260b8e87e30d543f6b9fba9f64c5a345224
  Stored in directory: /root/.cache/pip/wheels/3e/31/09/fa59cef12cdcfecc627b3d24273699f390e71828921b2cbba2
Successfully built pathtools


In [ ]:
import torch
import pandas as pd
import wandb 


In [ ]:
from pygooglenews import GoogleNews
gn = GoogleNews()

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification 
tokenizer = AutoTokenizer.from_pretrained("ProsusAI/finbert")
model = AutoModelForSequenceClassification.from_pretrained("ProsusAI/finbert")

Downloading:   0%|          | 0.00/252 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/758 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/418M [00:00<?, ?B/s]

In [ ]:
# search for the best matching articles that mention LUNA and 
# do not mention BTC (over the past 6 month)
#search = gn.search('LUNA -BTC', when = '6m')


In [ ]:
#List of cryptos to capture headlines for 
#fetch headlines from list of cryptos

headline_list = []
crypto_list = []

cryptos = ['bitcoin', 'ethereum','fantom','luna','avalanche','solana']
for crypto in cryptos:
  response = gn.search(crypto)
  for r in response['entries']:
    headline_list.append(r['title'])
    crypto_list.append(crypto)

    

In [ ]:
#Setting up project in wandb to push results into a wandb Table for easy visualization & analysis
#MUST HAVE SIGNED IN "https://wandb.ai/home" for API access code


wandb.init(project='Crypto Sentiment Analysis')
headline_table = wandb.Table(columns=["Headline","Crypto", "Postive", "Negative","Neutral"])

<IPython.core.display.Javascript object>

wandb: You can find your API key in your browser here: https://wandb.ai/authorize


wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [ ]:
#Peek at headlines captured 

for h in headline_list:
  print(h)

Bitcoin tumbles more than 50% below its all-time high as crypto plunges again - CNN
Ethereum and Bitcoin Prices Are Tanking. Here’s Why One Expert Thinks Bitcoin Could Drop Even Lower - NextAdvisor
Where To Buy Bitcoin At 26% Off Regular Price - Forbes
Bitcoin’s value nears $30,000 mark as Luna Foundation Guard liquidates wallet - TechCrunch
El Salvador Acquires 500 Additional Bitcoin Amid Market Drop - CoinDesk
The big question: is Bitcoin worth anything? - POLITICO
MicroStrategy Stock Is Crumbling as Bitcoin Slides and Rates Rise - Barron's
Bitcoin retests key $30K support zone as data highlights BTC whale accumulation - Cointelegraph
As BTC Slides Toward Resistance, the Chance of a Rare Triple Top Formation Comes Into Play – Markets and Prices Bitcoin News - Bitcoin News
You will soon be able to use Bitcoin to buy Gucci - CNN
First Mover Asia: BitMEX Saga Leaves Lingering Regulatory Questions, Bitcoin Falls - CoinDesk
What To Expect After President Biden’s Executive Order On Bitcoin

In [ ]:
#Loop through crypto list capturing sentiment prediction into a 3 column matrix for Positive, Negative, Neutral  
def chunkList(list,n):
  for i in range(0, len(list),n):
    yield list[i:i+n]

n = 0
chunkSize = 100
model.eval()
for lines, cryptos in zip(chunkList(headline_list,chunkSize), chunkList(crypto_list,chunkSize)):
  input = tokenizer(lines,padding=True, truncation=True, return_tensors='pt')
  outputs = model(**input)
  prediction = torch.nn.functional.softmax(outputs.logits, dim=-1)
  print(f"{n+1}/{int(len(headline_list)/chunkSize)}")
  for headline,crypto,pos,neg,neutr in zip(lines, cryptos, prediction[:,0].tolist(),prediction[:,1].tolist(),prediction[:,2].tolist()):
    headline_table.add_data(headline,crypto,pos,neg,neutr)
  n = n + 1

1/6
2/6
3/6
4/6
5/6
6/6


In [ ]:
#Push model predictions into a wandb table

wandb.run.log({"Crypto Sentiment Analysis Table": headline_table})
wandb.run.finish()